### RNA folding kinetics control riboswitch sensitivity in vivo 
#### David Z. Bushhouse1,3 , Jiayu Fu1,3, & Julius B. Lucks1,2,3,4,5,6* 

 

1 Interdisciplinary Biological Sciences Graduate Program, Northwestern University, Evanston, Illinois 60208, USA 

2 Department of Chemical and Biological Engineering, Northwestern University, Evanston, Illinois 60208, USA 

3 Center for Synthetic Biology, Northwestern University, Evanston, Illinois 60208, USA 

4 Center for Water Research, Northwestern University, Evanston, Illinois 60208, USA 

5 Center for Engineering Sustainability and Resilience, Northwestern University, Evanston, Illinois 60208, USA 

6 International Institute for Nanotechnology, Northwestern University, Evanston, Illinois 60208, USA 


* To whom correspondence should be addressed. Tel: 1-847-467-2943; Email: jblucks@northwestern.edu  

## Processing sequencing data and estimating mean fluorescence for each loop sequence at each ligand condition
This code is part 1 of the FACS-seq analysis pipeline. Ensure that sequencing reads (fastq) from SRA Deposition PRJNA1087340 and cell count metadata files are in the directory.

### Install the necessary packages and set up file path

In [ ]:
import os
import glob
import re
import pandas as pd
import numpy as np
import math
import csv
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
import shutil
from Bio import SeqIO
from Bio.SeqIO.QualityIO import FastqGeneralIterator

In [ ]:
## Specify were the raw data is read and where the analysis files are stored.
# Define the directory where your command (script) files reside
command_path = 'PATH-TO-SRA-DEPOSITION'
# Define the directory where your data files are located
data_path = 'PATH-TO-SRA-DEPOSITION'
# Define the analysis subfolder and create it if it doesn't exist
analysis_path = os.path.join(data_path, 'Analysis')
if not os.path.exists(analysis_path):
    os.makedirs(analysis_path)

# Define the experimental prefix
experimental_prefix = 'Rep1'
experimental_reads = experimental_prefix + "*.fastq"

# Ensure that the Cell Count files are in the same directory
cell_counts = experimental_prefix + '_CellCounts.txt'

# Define the loop length range
min_loop_length = 7
max_loop_length = 7

#### Quality threshold and removal of non-matching PE reads

In [ ]:
# Define the markers for R1
#start_marker_r1 is the sequence in the ZTP riboswitch upstream of the 7nt Loop sequence from Read 1 (top strand 5'-3')
start_marker_r1 = "AAAAAGCCGACCGTCTGGGC"
#end_marker_r1 is the sequence in the ZTP riboswitch downstream of the 7nt Loop sequence from Read 1(top strand 5'-3')
end_marker_r1 = "CCTGGATTGCGTCGGCTTTT"
# Define the markers for R2
#start_marker_r2 is the sequence in the ZTP riboswitch upstream of the 7nt Loop sequence from Read 2 (bottom strand 5'-3')
start_marker_r2 = "AAAAGCCGACGCAATCCAGG"
#end_marker_r2 is the sequence in the ZTP riboswitch downstream of the 7nt Loop sequence from Read 2(top strand 5'-3')
end_marker_r2 = "GCCCAGACGGTCGGCTTTTT"
## These should be reverse compliment of each other

# Function to generate reverse complement of Loop sequence for match processes
def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'}
    return ''.join(complement[base] for base in reversed(seq))

# Function to filter out reads with low quality score
def filter_reads_by_qscore(read1_file, read2_file, output_read1, output_read2, threshold=30):
    with open(read1_file, 'r') as r1, open(read2_file, 'r') as r2, \
         open(output_read1, 'w') as o1, open(output_read2, 'w') as o2:
         
         while True:
            r1_lines = [r1.readline().strip() for _ in range(4)]
            r2_lines = [r2.readline().strip() for _ in range(4)]
            
            if not r1_lines[0] or not r2_lines[0]:  # End of file
                break

            r1_qual = r1_lines[3]
            r2_qual = r2_lines[3]
            
            if all(ord(ch) - 33 >= threshold for ch in r1_qual) and \
               all(ord(ch) - 33 >= threshold for ch in r2_qual):
                o1.write("\n".join(r1_lines) + "\n")
                o2.write("\n".join(r2_lines) + "\n")

# Function extract the section with the loop sequence
def extract_sequence_between_markers(seq, start_marker, end_marker):
    start_idx = seq.find(start_marker)
    end_idx = seq.find(end_marker)
    
    if start_idx != -1 and end_idx != -1 and start_idx < end_idx:
        return seq[start_idx + len(start_marker):end_idx]
    return None

# Function to see if for the same read R1 and R2 matches, if not, discard.
# Also generate a count of how many reads are kept and are discarded.
def filter_by_sequence_match(input_read1_file, input_read2_file, output_read1_file, output_read2_file):
    with open(input_read1_file, 'r') as r1, open(input_read2_file, 'r') as r2, \
         open(output_read1_file, 'w') as o1, open(output_read2_file, 'w') as o2:

         no_marker_r1_count = 0
         no_marker_r2_count = 0
         no_match_count = 0
         total_count = 0

         while True:
            r1_lines = [r1.readline().strip() for _ in range(4)]
            r2_lines = [r2.readline().strip() for _ in range(4)]

            if not r1_lines[0] or not r2_lines[0]:
                break

            total_count += 1

            sequence_r1 = extract_sequence_between_markers(r1_lines[1], start_marker_r1, end_marker_r1)
            sequence_r2_extracted = extract_sequence_between_markers(r2_lines[1], start_marker_r2, end_marker_r2)
            sequence_r2 = reverse_complement(sequence_r2_extracted) if sequence_r2_extracted else None
            
            if not sequence_r1:
                no_marker_r1_count += 1
            if not sequence_r2_extracted:
                no_marker_r2_count += 1
            if not sequence_r1 or not sequence_r2_extracted:
                continue
            
            if sequence_r1 != sequence_r2:  
                no_match_count += 1
                continue

            o1.write("\n".join(r1_lines) + "\n")
            o2.write("\n".join(r2_lines) + "\n")

         print(f"Total reads: {total_count}")
         print(f"Reads without markers in R1: {no_marker_r1_count}")
         print(f"Reads without markers in R2: {no_marker_r2_count}")
         print(f"Reads with markers but not matching: {no_match_count}")

# Combine everything needed for the function call.
def process_files(directory_path):
    files = sorted([f for f in os.listdir(directory_path) if f.endswith('.fastq')])

    paired_files = {}
    for file in files:
        pieces = file.split('_')
        prefix = pieces[1] + '_' + pieces[2] + '_' + pieces[3]
        if "R1" in file:
            paired_files[prefix] = paired_files.get(prefix, {})
            paired_files[prefix]['R1'] = file
        elif "R2" in file:
            paired_files[prefix] = paired_files.get(prefix, {})
            paired_files[prefix]['R2'] = file

    for prefix, pair in paired_files.items():
        if 'R1' in pair and 'R2' in pair:
            read1_file_path = os.path.join(directory_path, pair['R1'])
            read2_file_path = os.path.join(directory_path, pair['R2'])
            
            filtered_read1_file_path = os.path.join(directory_path, f"filtered_{prefix}_R1.fastq")
            filtered_read2_file_path = os.path.join(directory_path, f"filtered_{prefix}_R2.fastq")
            filter_reads_by_qscore(read1_file_path, read2_file_path, filtered_read1_file_path, filtered_read2_file_path)
            
            # Move the original files to Raw_Data
            raw_data_dir = os.path.join(directory_path, 'Raw_Data')
            if not os.path.exists(raw_data_dir):
                os.makedirs(raw_data_dir)
            shutil.move(read1_file_path, raw_data_dir)
            shutil.move(read2_file_path, raw_data_dir)
            print("Raw data move complete.")

            matched_read1_file_path = os.path.join(directory_path, f"matched_{prefix}_R1.fastq")
            matched_read2_file_path = os.path.join(directory_path, f"matched_{prefix}_R2.fastq")
            filter_by_sequence_match(filtered_read1_file_path, filtered_read2_file_path, matched_read1_file_path, matched_read2_file_path)
            
            # Move the filtered files to QualityFiltered_Data
            quality_filtered_data_dir = os.path.join(directory_path, 'QualityFiltered_Data')
            if not os.path.exists(quality_filtered_data_dir):
                os.makedirs(quality_filtered_data_dir)
            shutil.move(filtered_read1_file_path, quality_filtered_data_dir)
            shutil.move(filtered_read2_file_path, quality_filtered_data_dir)
            print("Filtered data move complete.")

            # Move the matched files to matched_R2
            matched_R2_dir = os.path.join(directory_path, 'matched_R2')
            if not os.path.exists(matched_R2_dir):
                os.makedirs(matched_R2_dir)
            shutil.move(matched_read2_file_path, matched_R2_dir)
            print("Matched data move complete.")


# Call the function
process_files(data_path)


#### Calculating read counts for each loop based on ligand condition and bin

In [ ]:
# Initializes an empty DataFrame
df_total = pd.DataFrame()

# Open a text file in analysis_path to write the report
report_filename = f"ReportRawDataInSequence_Bins_{experimental_prefix}.txt"
with open(os.path.join(analysis_path, report_filename), 'w') as report_file:

    # Here specify which replicate is being 
    for filename in glob.glob(os.path.join(data_path, 'matched*.fastq')):
        count = 0
        total_len = 0
        seqs = []

        with open(filename, 'r') as in_handle:
            for title, seq, qual in FastqGeneralIterator(in_handle):
                count += 1
                total_len += len(seq)
                seqs.append(seq)

        info_string = "%i records with total sequence length %i" % (count, total_len)
        print(info_string)
        report_file.write(info_string + '\n')

        loops = []
        correct = 0
        fail = 0
        total = 0
        
        for seq in seqs:
            loop = ''
            if 'CATATTATCTTATATGCCACAAAAAGCCGACCGTCTGGGC' in seq:
                loop = seq.split('CATATTATCTTATATGCCACAAAAAGCCGACCGTCTGGGC')[1].split('GCCTGGATTGCGTCGGCTTTT')[0]
                if min_loop_length <= len(loop) <= max_loop_length:  # Check if loop length is within the range
                    if 'N' not in loop:
                        loops.append(loop)
                        correct += 1
                    else:
                        fail += 1
                else:
                    fail += 1
                total += 1

        counts = dict()
        name = os.path.basename(filename).split('_')[3].split('S')[1]
        col_name = int(name)

        for i in sorted(loops):
            counts[i] = counts.get(i, 0) + 1

        df = pd.DataFrame.from_dict(counts, orient='index', dtype=None, columns=[col_name])

        if df_total.empty:
            df_total = df
        else:
            df_total = pd.concat([df_total, df], axis=1)

        df_total = df_total.sort_index(axis=1)

# Rename the first column as "Loop"
df_total = df_total.rename_axis("Loop", axis=1)

# Saves df_total to a .csv file in analysis_path
csv_filename = f"RawData_Bins_{experimental_prefix}.csv"
df_total.to_csv(os.path.join(analysis_path, csv_filename))

#### Normalizing read counts

In [ ]:
csv_file_path = os.path.join(analysis_path, f'RawData_Bins_{experimental_prefix}.csv')

# Read in the original CSV file to a DataFrame
df_original = pd.read_csv(csv_file_path)

# Read in the CSV file to a DataFrame for calculating the normalization ratios
df = pd.read_csv(csv_file_path)

# Exclude the first row and calculate the column-wise sum
column_sums = df.iloc[1:].sum()

# Create a new DataFrame to store the column sums
df_sums = pd.DataFrame(column_sums, columns=['Sum'])

# Rename the column
df_sums = df_sums.rename(columns={'Sum': 'SUM of reads'})

# Drop the first row
df_sums = df_sums.drop(df_sums.index[0])

# Read 'Cells counted' values from 'CellNumber.txt'
with open(os.path.join(data_path, cell_counts), 'r') as f:
    cells_counted_values = [int(line.strip()) for line in f if line.strip() and line.strip().isdigit()]

# Add 'Cells counted' column and set the values for rows 2-21
df_sums.loc[df_sums.index[1:21], 'Cells counted'] = cells_counted_values

# Create 'Normalized reads' column
max_sum_of_reads = df_sums['SUM of reads'].max()
df_sums['Normalized reads'] = df_sums['SUM of reads'] / max_sum_of_reads

# Create 'Normalized cells' column
max_cells_counted = df_sums['Cells counted'].max()
df_sums['Normalized cells'] = df_sums['Cells counted'] / max_cells_counted

# Create 'Normalized Ratios' column
df_sums['Normalized Ratios'] = df_sums['Normalized reads'] / df_sums['Normalized cells']

print(df_sums)

# Normalize the original DataFrame using the calculated 'Normalized Ratios'
for column in df_original.columns[1:]:
    df_original[column] = df_original[column] / df_sums.at[column, 'Normalized Ratios']

# Construct the output filename using the experimental_suffix
output_filename = os.path.join(analysis_path, f'NormalizedData_{experimental_prefix}.csv')

# Write the normalized DataFrame to the new CSV file
df_original.to_csv(output_filename, index=False)


#### Removing sparse data before calculating mean fluorescence estimate

In [ ]:
# Define the path to the CSV file and the analysis path
csv_file_path = os.path.join(analysis_path, f'NormalizedData_{experimental_prefix}.csv')

# Read the data into a DataFrame
df = pd.read_csv(csv_file_path)

# Define the column ranges to check
column_ranges = [
    (2, 5),
    (6, 9),
    (10, 13),
    (14, 17),
    (18, 21)
]

# Function to count non-zero and non-blank datapoints in a given range of columns
def count_valid_data(df, start_col, end_col):
    cols = [f'{i}' for i in range(start_col, end_col + 1)]
    valid_data = df[cols].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert to numeric and treat errors as NaN, then fill NaN with 0
    return (valid_data != 0).sum(axis=1)

# Apply the function to each range and determine if the row meets the criteria for each range
for start_col, end_col in column_ranges:
    df[f'count_{start_col}_{end_col}'] = count_valid_data(df, start_col, end_col)

# Define a condition for 'Good Data'
# Check if each range count meets the criteria for each row
conditions = [(df[f'count_{start_col}_{end_col}'] >= 3) for start_col, end_col in column_ranges]
df['meets_criteria'] = pd.concat(conditions, axis=1).all(axis=1)

# Split the data into 'GoodData' and 'BadData'
good_data = df[df['meets_criteria']].fillna(0)  # Fill blank cells with 0
bad_data = df[~df['meets_criteria']]

# Output filenames
good_data_output_filename = os.path.join(analysis_path, f'GoodDataforLogTransform_{experimental_prefix}.csv')
bad_data_output_filename = os.path.join(analysis_path, f'BadDataforLogTransform_{experimental_prefix}.csv')

# Save to CSV files
good_data.to_csv(good_data_output_filename, index=False)
bad_data.to_csv(bad_data_output_filename, index=False)


#### Calculating mean fluorescence estimate for each loop in each ligand condition

Average of read counts in each bin is weighted by the geometric mean fluorescence of each bin:

Gate    | Range          | Geo Mean |
--------|----------------|----------|
Bin 1   | 2767 - 9114    | 5021     |
Bin 2   | 9115 - 28952   | 16245    |
Bin 3   | 28953 - 85559  | 49771    |
Bin 4   | 85560 - 262143 | 149762   |


In [ ]:
csv_file_path = os.path.join(analysis_path, f'GoodDataforLogTransform_{experimental_prefix}.csv')

# Read in the normalized data CSV file to a DataFrame
df_normalized = pd.read_csv(csv_file_path)

# Create a new DataFrame
df_0mM = pd.DataFrame()

# Add the original first column to the new DataFrame
df_0mM[df_normalized.columns[0]] = df_normalized.iloc[:, 0]

# Apply the provided equation to the first column
df_0mM['0mM'] = (np.log10(5021) * df_normalized.iloc[:, 2] +
                 np.log10(16245) * df_normalized.iloc[:, 3] +
                 np.log10(49771) * df_normalized.iloc[:, 4] +
                 np.log10(149762) * df_normalized.iloc[:, 5]) / df_normalized.iloc[:, 2:6].sum(axis=1)

# Apply the provided equation to each row for 0_01mM, 0_1mM, 0_32mM, and 1mM
for i, concentration in zip(range(6, 22, 4), ['0_01mM', '0_1mM', '0_32mM', '1mM']):
    df_0mM[concentration] = (np.log10(5021) * df_normalized.iloc[:, i] +
                             np.log10(16245) * df_normalized.iloc[:, i+1] +
                             np.log10(49771) * df_normalized.iloc[:, i+2] +
                             np.log10(149762) * df_normalized.iloc[:, i+3]) / df_normalized.iloc[:, i:i+4].sum(axis=1)

# Construct the output filename using the extracted name_part
output_filename = os.path.join(analysis_path, f'LogTransformed_{experimental_prefix}.csv')

# Write the DataFrame to the new CSV file
df_0mM.to_csv(output_filename, index=False)


In [ ]:
# Specify the path to your LogTransformed data CSV file
csv_file_path = os.path.join(analysis_path, f'LogTransformed_{experimental_prefix}.csv')

# Read in the LogTransformed.csv
df_log_transformed = pd.read_csv(csv_file_path)

# Create a new DataFrame and copy the original first column
df_multiplied = pd.DataFrame()
df_multiplied[df_log_transformed.columns[0]] = df_log_transformed.iloc[:, 0]

# Apply the transformation to the rest of the DataFrame
df_multiplied[df_log_transformed.columns[1:]] = 10 ** df_log_transformed.iloc[:, 1:]

# Construct the output filename using the experimental_suffix
output_filename = os.path.join(analysis_path, f'MultipliedData_{experimental_prefix}.csv')

# Write out to the new CSV file
df_multiplied.to_csv(output_filename, index=False)



#### Generating summary data

In [ ]:
data_stats_df = pd.read_csv(os.path.join(analysis_path, f'RawData_Bins_{experimental_prefix}.csv'))

def count_rows_with_data(csv_file):
    try:
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(csv_file)

        # Count the number of non-null values for each column
        counts = df.count()

        # Calculate the percentage of non-null values for each column relative to the first row
        first_row_count = counts.iloc[0]
        percentages = (counts / first_row_count) * 100

        # Create a new DataFrame with the counts and column names
        data_stats_df = pd.DataFrame({"Sample Name": counts.index, 
                                      "Number of Loops recovered": counts.values,
                                      "Percentage of loops recovered": percentages.values})

        # Print the statistics to the console
        print("DataStats:")
        print(data_stats_df)

        # Construct the output filename using the experimental_suffix
        output_filename = os.path.join(command_path, f'DataStats_{experimental_prefix}.csv')

        # Write the DataFrame to the new CSV file
        data_stats_df.to_csv(output_filename, index=False)

        print(f"\n{output_filename} file created successfully.")


    except FileNotFoundError:
        print(f"Error: File '{csv_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example usage:
csv_file_path = os.path.join(analysis_path, f'RawData_Bins_{experimental_prefix}.csv')
count_rows_with_data(csv_file_path)


This is the end of Part 1 of the code. Repeat this with data from the second replicate by changing `experimental_prefix`.